## 1.) Dependencies and Setup

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from random import sample

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## 2.) Generate Cities List

In [10]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

640

In [15]:
# Select a random sample of 500 cities from list generated in above cell
cities_sample = sample(cities,500)
cities_sample

['staryy nadym',
 'suez',
 'tobol',
 'limon',
 'waiouru',
 'livani',
 'kizema',
 'hasaki',
 'lisala',
 'bama',
 'jiuquan',
 'at-bashi',
 'birao',
 'port augusta',
 'necochea',
 'atar',
 'katsuura',
 'kaura namoda',
 'narsaq',
 'busselton',
 'bereda',
 'opuwo',
 'beberibe',
 'kemijarvi',
 'marsh harbour',
 'eirunepe',
 'derzhavinsk',
 'arraial do cabo',
 'piacabucu',
 'komsomolsk-na-amure',
 'garowe',
 'fortuna',
 'constitucion',
 'flinders',
 'talisayan',
 'boende',
 'lagoa vermelha',
 'taolanaro',
 'mount isa',
 'new norfolk',
 'teshie',
 'kavaratti',
 'port alfred',
 'ban phaeo',
 'dikson',
 'bud',
 'mackenzie',
 'havoysund',
 'biloela',
 'raga',
 'tambovka',
 'nanortalik',
 'cherskiy',
 'morant bay',
 'camacari',
 'celestun',
 'mahajanga',
 'cotonou',
 'rungata',
 'sakakah',
 'avarua',
 'luderitz',
 'nyurba',
 'torbay',
 'carnarvon',
 'yagodnoye',
 'kilindoni',
 'awbari',
 'buraydah',
 'huambo',
 'mataura',
 'eureka',
 'san carlos de bariloche',
 'bathsheba',
 'coquimbo',
 'koungou'

## 3.) Generate API Calls to Create Dataframe

In [22]:
# Create API calls to pull current weather data
base_url = "http://api.openweathermap.org/data/2.5/weather?q="
units = 'imperial'

city_name = 'busselton'


query_url = f'{base_url}{city_name}&appid={weather_api_key}&units={units}'

response = requests.get(query_url).json()
# response

# latitude = response['coord']['lat']
# longitude = response['coord']['lon']
# max_temp = response['main']['temp_max']
# humidity = response['main']['humidity']
# cloudiness = response['clouds']['all']
# wind_speed = response['wind']['speed']
# country = response['sys']['country']
# date = response['dt']

# print(latitude)
# print(longitude)
# print(max_temp)
# print(humidity)
# print(cloudiness)
# print(wind_speed)
# print(country)
# print(date)


-33.65
115.3333
54
90
11
8.81
AU
1618760962


In [25]:
# Loop through list of cities in sets of 50, append records to dataframe
# Sleep for 30 seconds in between each set

# Create blank dataframe with relevant fields, so that records from loop can be added in
column_names = ['City','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed','Country','Date']
city_weather_df = pd.DataFrame(columns=column_names)

# ---------- BEGIN LOOP FOR BULK API CALL ----------
# Set starting index # for city loop, so that you continue through each city list as you progres to subsequent set
city_loop_index = 0
for loop_set in range(0,10):
    print(f'--------- Set {loop_set + 1} of 10 ---------- ')
    # Loop through cities and make API calls
    for city_index in range(0,50):
        # Add city_loop_index variable to ensure that loop continues through list of 500
        final_index = city_index + city_loop_index
        # Pull current city
        city_name = cities_sample[final_index]
        # Generate query string
        query_url = f'{base_url}{city_name}&appid={weather_api_key}&units={units}'
        # Make API Call
        response = requests.get(query_url).json()
        # Create try/except block to skip record if city is not found
        try:
            # Extract relevant data from response
            latitude = response['coord']['lat']
            longitude = response['coord']['lon']
            max_temp = response['main']['temp_max']
            humidity = response['main']['humidity']
            cloudiness = response['clouds']['all']
            wind_speed = response['wind']['speed']
            country = response['sys']['country']
            date = response['dt']
            # Create dictionary to append to df
            record_dict = {
                'City':city_name,
                'Lat':latitude,
                'Lng':longitude,
                'Max Temp':max_temp,
                'Humidity':humidity,
                'Cloudiness':cloudiness,
                'Wind Speed':wind_speed,
                'Country':country,
                'Date':date
            }
            # Append record dict to city_weather_df
            city_weather_df = city_weather_df.append(record_dict,ignore_index=True)
            print(f'City {city_name} found successfully as record {city_index + 1} of set {loop_set + 1}.')
        except (KeyError,IndexError):
            print(f'City {city_name} not found. Skipping...')
    # ---------- Outer Loop ----------
    # Add 50 to city_loop_index to ensure that inner loop continues with next set of 50 cities
    city_loop_index += 50
    # Sleep for 30 seconds in between each set
    time.sleep(30)

print('---------- Data Retrieval Complete ----------')
            

--------- Set 1 of 10 ---------- 
City staryy nadym found successfully as record 1 of set 1.
City suez found successfully as record 2 of set 1.
City tobol found successfully as record 3 of set 1.
City limon found successfully as record 4 of set 1.
City waiouru found successfully as record 5 of set 1.
City livani found successfully as record 6 of set 1.
City kizema found successfully as record 7 of set 1.
City hasaki found successfully as record 8 of set 1.
City lisala found successfully as record 9 of set 1.
City bama found successfully as record 10 of set 1.
City jiuquan found successfully as record 11 of set 1.
City at-bashi found successfully as record 12 of set 1.
City birao found successfully as record 13 of set 1.
City port augusta found successfully as record 14 of set 1.
City necochea found successfully as record 15 of set 1.
City atar found successfully as record 16 of set 1.
City katsuura found successfully as record 17 of set 1.
City kaura namoda found successfully as record

City tudela found successfully as record 48 of set 3.
City dickinson found successfully as record 49 of set 3.
City praya found successfully as record 50 of set 3.
--------- Set 4 of 10 ---------- 
City bethanien found successfully as record 1 of set 4.
City barranco found successfully as record 2 of set 4.
City prince rupert found successfully as record 3 of set 4.
City bom jesus found successfully as record 4 of set 4.
City canapi found successfully as record 5 of set 4.
City laguna found successfully as record 6 of set 4.
City amahai found successfully as record 7 of set 4.
City luganville found successfully as record 8 of set 4.
City terrace found successfully as record 9 of set 4.
City maniitsoq found successfully as record 10 of set 4.
City ilulissat found successfully as record 11 of set 4.
City alberton found successfully as record 12 of set 4.
City buala found successfully as record 13 of set 4.
City menomonie found successfully as record 14 of set 4.
City matay found successf

City byron bay found successfully as record 48 of set 6.
City leh found successfully as record 49 of set 6.
City sisimiut found successfully as record 50 of set 6.
--------- Set 7 of 10 ---------- 
City soledade found successfully as record 1 of set 7.
City sidhi found successfully as record 2 of set 7.
City wanning found successfully as record 3 of set 7.
City jamestown found successfully as record 4 of set 7.
City vozhega found successfully as record 5 of set 7.
City saskylakh found successfully as record 6 of set 7.
City tabas found successfully as record 7 of set 7.
City port macquarie found successfully as record 8 of set 7.
City tyukhtet found successfully as record 9 of set 7.
City pedernales found successfully as record 10 of set 7.
City grand centre not found. Skipping...
City chifeng found successfully as record 12 of set 7.
City seoul found successfully as record 13 of set 7.
City fairbanks found successfully as record 14 of set 7.
City riyadh found successfully as record 15

City khatanga found successfully as record 45 of set 9.
City sampang found successfully as record 46 of set 9.
City izhmorskiy found successfully as record 47 of set 9.
City adre found successfully as record 48 of set 9.
City vostok found successfully as record 49 of set 9.
City souillac found successfully as record 50 of set 9.
--------- Set 10 of 10 ---------- 
City shieli found successfully as record 1 of set 10.
City tawang found successfully as record 2 of set 10.
City mahebourg found successfully as record 3 of set 10.
City hailar found successfully as record 4 of set 10.
City verkhnetulomskiy found successfully as record 5 of set 10.
City anadyr found successfully as record 6 of set 10.
City columbus found successfully as record 7 of set 10.
City shanhetun found successfully as record 8 of set 10.
City sorvag not found. Skipping...
City itamaraju found successfully as record 10 of set 10.
City arani found successfully as record 11 of set 10.
City kondinskoye found successfully a

In [26]:
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,humidity
0,staryy nadym,65.6118,72.6842,-0.71,NaN,87,9.82,RU,1618763663,94.0
1,suez,29.9737,32.5263,82.18,NaN,0,8.57,EG,1618763703,27.0
2,tobol,52.6844,62.5794,39.33,NaN,9,22.75,KZ,1618763703,56.0
3,limon,10.0000,-83.0333,86.00,NaN,0,4.61,CR,1618763703,66.0
4,waiouru,-39.4833,175.6667,37.47,NaN,28,4.05,NZ,1618763704,95.0
...,...,...,...,...,...,...,...,...,...,...
465,kachiry,53.0675,76.1036,35.20,NaN,97,21.72,KZ,1618764066,67.0
466,lavrentiya,65.5833,-171.0000,18.34,NaN,96,2.82,RU,1618763824,97.0
467,tabuk,17.4189,121.4443,69.89,NaN,97,9.84,PH,1618763834,89.0
468,kargasok,59.0556,80.8572,15.42,NaN,100,19.69,RU,1618763864,77.0
